In [1]:
import os

In [2]:
from tensorflow.keras.preprocessing import image

In [3]:
import cv2

In [4]:
categories = ['with_mask','without_mask']

In [5]:
data = []
for category in categories:
    path = os.path.join('train',category)
    
    label = categories.index(category)
    
    for file in os.listdir(path):
        
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        
        data.append([img,label])

In [6]:
len(data)

1508

In [7]:
import random

In [8]:
random.shuffle(data)

In [9]:
X = []
y = []

for features,label in data:
    X.append(features)
    y.append(label)

In [10]:
len(X)

1508

In [11]:
len(y)

1508

In [12]:
import numpy as np

In [13]:
X = np.array(X)
y = np.array(y)

In [14]:
X.shape   #(No.of images,width,height,No.of channels)

(1508, 224, 224, 3)

In [15]:
y.shape

(1508,)

In [16]:
X = X/255  #Standardize the X

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [18]:
X_train.shape

(1206, 224, 224, 3)

In [19]:
X_test.shape

(302, 224, 224, 3)

In [20]:
from tensorflow.keras.applications.vgg16 import VGG16

In [21]:
vgg = VGG16()

In [22]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [23]:
from tensorflow.keras import Sequential

In [24]:
model = Sequential()

In [25]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [27]:
for layer in model.layers:
    layer.trainable=False

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [29]:
from tensorflow.keras.layers import Dense

In [30]:
model.add(Dense(1,activation='sigmoid'))

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [32]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [33]:
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
38/38 [==============================] - 1011s 27s/step - loss: 0.6773 - accuracy: 0.6235 - val_loss: 0.5154 - val_accuracy: 0.7219
Epoch 2/5
38/38 [==============================] - 962s 25s/step - loss: 0.3938 - accuracy: 0.8715 - val_loss: 0.3626 - val_accuracy: 0.8742
Epoch 3/5
38/38 [==============================] - 933s 25s/step - loss: 0.2853 - accuracy: 0.9279 - val_loss: 0.3003 - val_accuracy: 0.8974
Epoch 4/5
38/38 [==============================] - 942s 25s/step - loss: 0.2333 - accuracy: 0.9395 - val_loss: 0.2580 - val_accuracy: 0.9139
Epoch 5/5
38/38 [==============================] - 942s 25s/step - loss: 0.2048 - accuracy: 0.9494 - val_loss: 0.2247 - val_accuracy: 0.9106


In [41]:
cap = cv2.VideoCapture(0)

In [54]:
while True:
    
    ret, frame = cap.read()
    
    # call the detection method
    img = cv2.resize(frame,(224,224))
    
    y_pred = detect_face_mask(img)
    
    co_ord = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
    
    for x,y,w,h in co_ord:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
    
    if y_pred == 0:
        draw_label(frame,"Mask",(30,30),(0,255,0))
    else:
        draw_label(frame,"No Mask",(30,30),(0,0,255))
    
    cv2.imshow("window",frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cv2.destroyAllWindows()

In [44]:
def detect_face_mask(img):
    
    y_pred = model.predict_classes(img.reshape(1,224,224,3))
    
    return y_pred[0][0]

In [46]:
def draw_label(img,text,pos,bg_color):
    
    text_size = cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    
    end_x = pos[0] + text_size[0][0] + 2
    end_y = pos[1] + text_size[0][1] - 2
    
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

In [48]:
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [51]:
def detect_face(img):
    co_ord = haar.detectMultiScale(img)
    return co_ord